In [1]:
#### label 정보
#### (배경 - 0 / 몸통 - 1 / 오른손 - 2 / 왼손 - 3 / 왼발 - 4 / 오른발 - 5 / 오른쪽 허벅지 - 6 / 왼쪽 허벅지 - 7 / 오른쪽 종아리 - 8 / 왼쪽 종아리 - 9 / 왼쪽 팔 - 10 / 
#### 오른쪽 팔 - 11 / 왼쪽 전완 - 12 / 오른쪽 전완 - 13 / 머리 - 14)
#### 해당 값이 RGB 값임 ex) 배경 0,0,0
### mask 확장자 png input image 확장자 jpg 해깔리지 말것

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)
import mmcv
import os.path as osp

1.9.0 True
0.18.0


In [3]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
from custom_data_pipeline import CutOut,RandomAffine,CustomRandomFlip

In [4]:
classes = ('bg', 'body', 'right_hand', 'left_hand', 'left_leg', 'right_reg', 'right_thigh', 'left_thigh','right_calf','left_calf'
           ,'left_arm','right_arm','left_forearm','right_forearm','head')
## ver2용 추가 
flip_pair=[[2,3],[4,5],[6,7],[8,9],[10,11],[12,13]]  ##flip pair

In [5]:
palette = [[0,0,0], [1,1,1], [2,2,2], [3,3,3],[4,4,4], [5,5,5], [6,6,6], [7,7,7], [8,8,8], [9,9,9], [10,10,10], [11,11,11], [12,12,12], [13,13,13], [14,14,14]]

In [6]:
from mmcv import Config

In [7]:
from mmseg.apis import set_random_seed
cfg = Config.fromfile('mmsegmentation/configs/hrnet/fcn_hr18_512x512_160k_ade20k.py')

In [8]:
cfg.dataset_type = 'body_seg'
cfg.data_root = '/root'


#hrnet으로 시도..
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = dict(type='BN', requires_grad=True)

# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 15

cfg.data.samples_per_gpu = 13
cfg.data.workers_per_gpu = 2

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (512, 512)

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(640, 425), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.0),
    ##### ver 2용 수정 #########
    dict(type='CustomRandomFlip', prob=0.5, flip_pair=flip_pair), #기존 flip과 동일하게 horizontal flip 하면서 pair를 맞춰주는 거만 추가함 
    dict(type='CutOut', n_holes=(0,4),cutout_shape=[(16,16),(32,32),(48,48)]),   ## 추가 (비율과 정해진 사이즈 중에 사이즈로 처리함)
    dict(type='RandomAffine'),                                                   ## 추가 
    ##### ver 2용 수정 #########
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(640, 425),
        img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = 'train2014'
cfg.data.train.ann_dir = 'train_mask'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = 'val2014'
cfg.data.val.ann_dir = 'val_mask'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'


cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = 'val2014'
cfg.data.test.ann_dir = 'val_mask'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/train_hrnet'

#ade20k로 pretrain 된 weight 사용 
cfg.load_from = 'pretrained/fcn_hr18_512x512_160k_ade20k.pth'

# 1epoch 대략 2600
cfg.runner.max_iters = 2600*200
cfg.log_config.interval = 1300
cfg.evaluation.interval = 2600
cfg.checkpoint_config.interval = 2600

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='open-mmlab://msra/hrnetv2_w18',
    backbone=dict(
        type='HRNet',
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        extra=dict(
            stage1=dict(
                num_modules=1,
                num_branches=1,
                block='BOTTLENECK',
                num_blocks=(4, ),
                num_channels=(64, )),
            stage2=dict(
                num_modules=1,
                num_branches=2,
                block='BASIC',
                num_blocks=(4, 4),
                num_channels=(18, 36)),
            stage3=dict(
                num_modules=4,
                num_branches=3,
                block='BASIC',
                num_blocks=(4, 4, 4),
                num_channels=(18, 36, 72)),
            stage4=dict(
                num_modules=3,
                num_branches=4,
                block='BASIC',
      

In [9]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class body_seg(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None


In [10]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES


# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

/opt/conda/lib/python3.7/site-packages/mmcv/utils/misc.py:334: UserWarning: "flip_ratio" is deprecated in `RandomFlip.__init__`, please use "prob" instead
  f'"{src_arg_name}" is deprecated in '
2021-11-10 03:36:56,447 - mmseg - INFO - Loaded 26437 images
/opt/conda/lib/python3.7/site-packages/mmseg/models/backbones/hrnet.py:318: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
2021-11-10 03:36:58,547 - mmseg - INFO - Loaded 1908 images
2021-11-10 03:36:58,547 - mmseg - INFO - load checkpoint from pretrained/fcn_hr18_512x512_160k_ade20k.pth
2021-11-10 03:36:58,548 - mmseg - INFO - Use load_from_local loader
2021-11-10 03:36:58,983 - mmseg - WARNING - The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([150, 270, 1, 1]) from checkpoint, the shape in current model is torch.Size([15, 270, 1, 1]).
siz

[                                                  ] 0/1908, elapsed: 0s, ETA:

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 2.9 task/s, elapsed: 648s, ETA:     0s

2021-11-10 04:24:10,512 - mmseg - INFO - per class results:
2021-11-10 04:24:10,514 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      |  91.3 |  99.2 |
|      body     | 68.36 | 78.81 |
|   right_hand  | 10.02 | 11.37 |
|   left_hand   |  0.16 |  0.16 |
|    left_leg   |  1.23 |  1.25 |
|   right_reg   |  0.6  |  0.6  |
|  right_thigh  |  7.48 |  8.31 |
|   left_thigh  | 29.13 | 50.16 |
|   right_calf  | 17.71 | 23.49 |
|   left_calf   |  4.07 |  4.3  |
|    left_arm   |  0.94 |  0.94 |
|   right_arm   | 30.11 | 52.97 |
|  left_forearm |  1.84 |  1.88 |
| right_forearm | 20.37 | 32.43 |
|      head     | 60.87 | 67.41 |
+---------------+-------+-------+
2021-11-10 04:24:10,515 - mmseg - INFO - Summary:
2021-11-10 04:24:10,515 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 88.45 | 22.95 | 28.89 |
+-------+-------+-------+
2021-11-10 04:24:10,519 - mmse

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 3.0 task/s, elapsed: 644s, ETA:     0s

2021-11-10 13:49:37,840 - mmseg - INFO - per class results:
2021-11-10 13:49:37,841 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 93.06 | 98.98 |
|      body     | 69.54 | 76.99 |
|   right_hand  | 24.19 | 39.83 |
|   left_hand   |  9.64 | 11.75 |
|    left_leg   | 14.98 |  18.3 |
|   right_reg   |  4.98 |  5.66 |
|  right_thigh  | 12.97 | 15.73 |
|   left_thigh  | 20.67 | 36.89 |
|   right_calf  |  1.07 |  1.12 |
|   left_calf   | 28.43 |  49.4 |
|    left_arm   | 11.17 | 12.89 |
|   right_arm   | 31.06 | 47.67 |
|  left_forearm |  8.55 |  10.2 |
| right_forearm | 34.04 | 60.87 |
|      head     | 61.19 |  75.1 |
+---------------+-------+-------+
2021-11-10 13:49:37,842 - mmseg - INFO - Summary:
2021-11-10 13:49:37,842 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 89.03 | 28.37 | 37.43 |
+-------+-------+-------+
2021-11-10 13:49:37,846 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 3.0 task/s, elapsed: 644s, ETA:     0s

2021-11-10 14:36:45,091 - mmseg - INFO - per class results:
2021-11-10 14:36:45,093 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      |  93.6 | 99.14 |
|      body     | 71.19 | 77.91 |
|   right_hand  | 31.71 | 57.56 |
|   left_hand   |  0.78 |  0.78 |
|    left_leg   | 24.47 | 35.14 |
|   right_reg   |  0.01 |  0.01 |
|  right_thigh  | 23.72 | 39.76 |
|   left_thigh  | 15.83 | 20.42 |
|   right_calf  |  5.99 |  8.26 |
|   left_calf   |  30.8 | 48.97 |
|    left_arm   |  3.73 |  3.84 |
|   right_arm   | 34.97 | 58.38 |
|  left_forearm |  1.83 |  1.89 |
| right_forearm | 33.67 | 65.03 |
|      head     | 65.39 | 72.67 |
+---------------+-------+-------+
2021-11-10 14:36:45,093 - mmseg - INFO - Summary:
2021-11-10 14:36:45,094 - mmseg - INFO - 
+------+-------+-------+
| aAcc |  mIoU |  mAcc |
+------+-------+-------+
| 89.5 | 29.18 | 39.32 |
+------+-------+-------+
2021-11-10 14:36:45,099 - mmseg - I

KeyboardInterrupt: 